## Initialization

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

import os, sys, pandas as pd

sys.path.append('/Users/alexisdevitre/Documents/code/hts-irradiation/')
sys.path.append('../../')

import hts_fitting as hts, hts_showcase as sc, seaborn as sns, hts_fitfunctions as ff
from uncertainties import ufloat
from scipy import constants, optimize
from scipy.interpolate import interp1d
import default_style

savedir = '/Users/alexisdevitre/Documents/GitHub/2025-devitre-PhD/chapter 6/figures/raw/'

def get_B_field(z=0):
    data = pd.read_csv('flux_density_norm_above_tape.txt', sep='\s+', skiprows=8, names=['y_m', 'field_T'])
    data['y_m'] -= data['y_m'].max()
    data['y_m'] = data.y_m.abs()
    data['field_T'] /= data['field_T'].max()
    return interp1d(data.y_m, data.field_T, fill_value=1., bounds_error=False, kind='cubic')(z)

## Computation

In [ ]:
# Constants
q = constants.electron_volt           # Proton charge (C)
m = constants.atomic_mass             # Proton mass (kg)
initial_energy = 1200e3 * q           # Initial kinetic energy in joules
energy_loss_per_m = 100e3 * q / 1e-6  # 100 keV/um in J/m
field_start_z = 0                     # Field starts at z0 (downward)
tape_start_z = 0.04                   # The particle starts at y = 4 cm above the tape where the field is 1% of the maximum

# Initial conditions
r = np.array([0.0, 0.0, 0.0])              # Start at z = 0
v_mag = np.sqrt(2 * initial_energy / m)
v = np.array([0.0, 0.0, v_mag])           # Initial velocity in -z

# Time step and max steps
max_steps, step, dt = 300000, 1, 1e-14 #1e-14  # s

# Preallocate trajectory array
trajectory = np.zeros((max_steps, 3))
trajectory[0] = r.copy()

# Simulation loop
kinetic_energies = []
while np.linalg.norm(v) > 0 and step < max_steps:

    KE = 0.5 * m * np.linalg.norm(v)**2

    # Energy loss
    dz = v[2] * dt
    if r[2] >= tape_start_z:
        KE -= energy_loss_per_m * abs(dz)
        
    if KE <= 0:
        break

    # Update velocity magnitude
    v_mag = np.sqrt(2 * KE / m)
    v_dir = v / np.linalg.norm(v)
    v = v_mag * v_dir

    B = np.array([get_B_field(r[2]), 0.0, 0.0])  # Field in x-direction

    # Lorentz force
    a = q * np.cross(v, B) / m

    # Euler update
    v += a * dt
    r += v * dt

    trajectory[step] = r.copy()
    step += 1
    #if step%1000 == 0:
    #    print(step)
    kinetic_energies.append(KE)
    
# Trim unused part of trajectory
trajectory = trajectory[:step]

## Plot

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 8))

ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)

x = trajectory[:, 1]*1e6 # m to um
y = trajectory[:, 2]*1e2 # m to cm
y = np.abs(y-y.max())-(y.max()-4)

ax[0].plot(x, y*1e4, linewidth=4, solid_capstyle='round')  # z vs y
ax[0].set_xlabel(r'Horizontal deflection [$\mu$m]')
ax[0].set_ylabel(r'Above the surface [$\mu$m]')
ax[0].set_ylim(0, 4e4)
ax[0].set_xlim(0, 179.5)

ax[1].plot(x, y*1e4, linewidth=4, solid_capstyle='round')  # z vs y
ax[1].set_xlabel(r'Horizontal deflection [$\mu$m]')
ax[1].set_ylabel(r'Below the surface [$\mu$m]')
ax[1].set_xlim(179.5, 179.9)
ax[1].set_ylim(-13, 0)
ax[1].set_yticks([-12, -9, -6, -3, 0])
ax[1].set_xticks([179.5, 179.7, 179.9])

fig.tight_layout()
#plt.savefig(savedir+'deflection.svg', format='svg', transparent=True, dpi=300)